In [9]:
import sys
sys.path.append('/home/hjy/SSF/')

import torch

from timm.models import create_model, safe_model_name, resume_checkpoint, load_checkpoint,\
    convert_splitbn_model, model_parameters
from timm.utils import *
from timm.loss import *

from timm.scheduler import create_scheduler
from timm.utils import ApexScaler, NativeScaler


from data import create_loader, create_dataset
from optim_factory import create_optimizer_v2, optimizer_kwargs

from models import vision_transformer, swin_transformer, convnext, as_mlp

import models.lora
from models.lora import LoRA_ViT_timm
## ADDED for Pruning
import torch_pruning as tp
from pruning.group_pruning import SSFScalePruner, BNScaleImportance


In [2]:
model = create_model(
        'vit_base_patch16_224_in21k',
        pretrained=True,
        num_classes=100,
        drop_rate=0,
        drop_path_rate=0,
        global_pool=None,
        bn_momentum=None,
        bn_eps=None,
        scriptable=False,
        tuning_mode='ssf')

In [17]:
img = torch.randn(2, 3, 224, 224)
pred = model.forward(img)
print(pred.shape)

torch.Size([2, 100])


In [3]:
print(model)
print(model.ssf_scale_1)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0, inplace=False)
      )
      (ls2): Identi

In [4]:
lora_vit = LoRA_ViT_timm(vit_model=model, r=4, num_classes=100)

In [5]:
img = torch.randn(2, 3, 224, 224)
pred = lora_vit.forward(img)
print(pred.shape)

torch.Size([2, 100])


In [11]:
model_path = '/data/hjy/SSF/vit_base_patch16_224_in21k/last.pth.tar'
checkpoint = torch.load(model_path)

In [14]:
model = create_model(
        'vit_base_patch16_224_in21k',
        pretrained=True,
        num_classes=100,
        drop_rate=0,
        drop_path_rate=0,
        global_pool=None,
        bn_momentum=None,
        bn_eps=None,
        scriptable=False,
        tuning_mode='ssf')

In [15]:
model_state_dict = checkpoint['state_dict']

model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [17]:
img = torch.randn(2, 3, 224, 224)
pred = model.forward(img)
print(pred.shape)

torch.Size([2, 100])


In [ ]:
print(model)